In [1]:
# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

import numpy as np
import pandas as pd

import pims
import trackpy as tp
import os

import matplotlib  as mpl 
import matplotlib.pyplot as plt 

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 6))
mpl.rc('image', cmap='gray')

In /home/mrinalmanu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mrinalmanu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mrinalmanu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/mrinalmanu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mrinalmanu/.local/lib/python3.6/site-packages/m

In [2]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import imutils


In [3]:
from scipy import ndimage
from skimage import morphology, util, filters


In [4]:
import cv2

In [5]:
cap = cv2.VideoCapture(r'mov.mp4')
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video = cv2.VideoWriter(r'the_blob.avi', fourcc, 25, size)

g = []

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = 0

while(1):
    frame_count+=1
    time = float(frame_count)/fps
    
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.convertScaleAbs(frame)
    
    params = cv2.SimpleBlobDetector_Params()
    params.blobColor = 0
    params.filterByColor = True
    params.minArea = 0
    #by area
    params.filterByArea = True
    params.minArea = 100
    

    ###############################3333
    
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.2
    
    params.minThreshold = 50;
    params.maxThreshold = 200;
    ver = (cv2.__version__).split('.')
    if int(ver[0]) < 3:
        detector = cv2.SimpleBlobDetector(params)
    else:
        detector = cv2.SimpleBlobDetector_create(params)
        
    keypoints = detector.detect(frame)
    
    pts = cv2.KeyPoint_convert(keypoints)
    g.append([pts, time])

    
    im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    
    if ret == True:
        video.write(im_with_keypoints)
        cv2.imshow('frame', im_with_keypoints)
    else:
        cap.release()
        break
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
        
        

In [235]:
gx = [list(x) for x in g]


In [245]:
gx = pd.DataFrame(g)
df = gx
df['time'] = df[1].astype(float)
dfx = df[0].astype("string").str.split(' ', 1, expand=True)
df = df.drop([0], axis = 1)
df = df.drop([1], axis = 1)

dfx[0] = pd.Series(dfx[0].astype("string").str.replace(r"[^\d.]+",""))
dfx[1] = pd.Series(dfx[1].astype("string").str.replace(r"[^\d.]+",""))

dfx

ndf = pd.concat([df, dfx], axis=1)
#ndf = ndf.apply(pd.to_numeric, errors='coerce')
ndf.columns = ['time', 'x', 'y']
ndf  = ndf.replace('NA',np.nan)
ndf = ndf.dropna()

In [135]:
#ndf = ndf.reset_index(drop=True, inplace=True)

In [246]:
ndf

,time,x,y
0,0.033525,374.52914,173.87468
1,0.067049,374.52386,173.75543
2,0.100574,374.5077,173.41017
3,0.134098,374.5167,173.39484
4,0.167623,374.5006,173.4291
...,...,...,...
777,26.082069,369.73523,180.27576
778,26.115593,369.7756,180.25716
779,26.149118,369.75735,180.20224
780,26.182642,369.7487,180.2012


In [238]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from hhpy.plotting import animplot

%matplotlib inline

In [247]:
type(ndf['y'])

pandas.core.series.Series

In [248]:
ndf = ndf.apply(pd.to_numeric, errors='coerce')

In [249]:
type(ndf['y'])

pandas.core.series.Series

In [269]:
ndf['time'] = pd.to_datetime(ndf['time'])

In [270]:
#v = pd.to_timedelta(ndf['time'],unit='s')
ndf['time']

0     1970-01-01 00:00:00.000000000
1     1970-01-01 00:00:00.000000000
2     1970-01-01 00:00:00.000000000
3     1970-01-01 00:00:00.000000000
4     1970-01-01 00:00:00.000000000
                   ...             
777   1970-01-01 00:00:00.000000026
778   1970-01-01 00:00:00.000000026
779   1970-01-01 00:00:00.000000026
780   1970-01-01 00:00:00.000000026
781   1970-01-01 00:00:00.000000026
Name: time, Length: 773, dtype: datetime64[ns]

In [272]:
animplot(data=ndf, x='x', y='y', t='time',xlim=(0, 500), ylim=(0, 500),  t_format='%f', ls='None', marker='o', markerfacecolor='None', markeredgecolor='red', mode='html')